# 🚀 Korpusverarbeitung – Annotation mit spaCy

## Hinweise zur Ausführung des Notebooks
Dieses Notebook kann auf unterschiedlichen Levels erarbeitet werden (siehe Abschnitt ["Technische Voraussetzungen"](../introduction/introduction_requirements)): 
1. Book-Only Mode
2. Cloud Mode: Dafür auf 🚀 klicken und z.B. in Colab ausführen.
3. Local Mode: Dafür auf Herunterladen ↓ klicken und ".ipynb" wählen. 

## Übersicht
Im Folgenden wird exemplarisch der Roman "Effi Briest" von Theodor Fontane (txt-Datei) mit der Bibliothek [spaCy](https://spacy.io) annotiert.

Es werden folgendene Schritte durchgeführt:
1. Einlesen des Texts
3. Worthäufigkeiten ohne echte Tokenisierung
   * Aufteilen des Texts in Wörter auf Grundlage von Leerzeichen
   * Abfrage von Häufigkeiten
4. Annotation mit spaCy
   * Laden des Sprachmodells
   * Analysekomponenten auswählen
   * Text annotieren
   * Worthäufigkeiten anzeigen
5. Annotation speichern
6. Prozess für das gesamte Korpus ausführen

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
  
<b>Voraussetzungen zur Ausführung des Jupyter Notebooks</b>
<ol>
<li> Installieren der Bibliotheken </li>
<li>2. Laden der Daten (z.B. über den Command `wget` (s.u.))</li>
<li>3. Pfad zu den Daten setzen</li>
</ol>
Zum Testen: Ausführen der Zelle "load libraries" und der Sektion "Einlesen des Texts". </br>
Alle Zellen, die mit 🚀 gekennzeichnet sind, werden nur bei der Ausführung des Noteboos in Colab / JupyterHub bzw. lokal ausgeführt. 
</details>

In [1]:
#  🚀 Install libraries 
! pip install tqdm pandas numpy spacy bokeh 

#  🚀 Load german language model for annotation
! python -m spacy download de_core_news_sm

fish: Unknown command: pip
fish: 
 pip install tqdm pandas numpy spacy bokeh
 ^~^
/Users/henny/.cache/uv/archive-v0/1FpovyLPEhPkEVNQhItrP/bin/python: No module named spacy


In [6]:
# load libraries 
import json
import typing
import requests
from pathlib import Path
from time import time
from collections import OrderedDict, Counter

from tqdm import tqdm
import pandas as pd
import numpy as np
import spacy

from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.models import CustomJS, TextInput, Div

## Einlesen des Texts
Um eine Datei mit Python bearbeiten zu können, muss die Datei zuerst ausgewählt, d.h der [Pfad](https://en.wikipedia.org/wiki/Path_(computing)) zur Datei wird gesetzt, und dann eingelesen werden. 

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Zuerst wird der Ordner angelegt, in dem die Textdateien gespeichert werden. Der Einfachheit halber wird die gleich Datenablagestruktur wie in dem <a href="https://github.com/quadriga-dk/Text-Fallstudie-1/tree/main">GitHub Repository</a>, in dem die Daten gespeichert sind, vorausgesetzt. </br>
Der Text wird aus GitHub heruntergeladen und in dem Ordner <i>../data/txt/</i> abgespeichert. </br>
Der Pfad kann in der Variable <i>text_path</i> angepasst werden. Die einzulesenden Daten müssen die Endung `.txt` haben. </br>
</details>

In [7]:
# 🚀 Create data directory path
corpus_dir = Path("../data/corpus-of-german-fiction-txt")
if not corpus_dir.exists():
    corpus_dir.mkdir()

In [ ]:
# 🚀 Load the txt file from GitHub 
! wget https://raw.githubusercontent.com/quadriga-dk/Text-Fallstudie-1/refs/heads/main/data/txt/SNP2719372X-19181015-0-0-0-0.txt -P ../data/txt

In [10]:
# set the path to file to be processed
text_path = Path("../data/corpus-of-german-fiction-txt/Theodor_Fontane_-_Effi_Briest_(1895).txt")

In [24]:
# read text and print some parts of the text
if text_path.is_file():
    text = text_path.read_text()
    print(f"Textauszug:\n {text[127:266]}")
else:
    print("The file path does not exist. Set the variable text_path to an existing path.")

Textauszug:
 In Front des schon seit Kurfürst Georg Wilhelm von der Familie von Briest
bewohnten Herrenhauses zu Hohen-Cremmen fiel heller Sonnenschein 


Im Textauszug ist erkennbar, dass der Text die Absätze aus dem Text einer Print-Ausgabe entsprechen. Das ist für die automatische Prozessierung mit **spaCy** irrelevant, da die Absätze (kodiert durch `\n`) nicht als semantische Einheit gesehen werden.

## Worthäufigkeiten ohne echte Tokenisierung

### Text in Wörter aufteilen
Der einfachste Weg einen Text automatisch in Wörter aufzuteilen, ist anzunehmen, dass Wörter durch Leerzeichen getrennt sind.

In [25]:
# split the text into words by space
words = text.split()

**Prüfen**: Wie sieht die Wortliste aus?

In [35]:
# print the 100th up the 120th words
words[7:79]

['In',
 'Front',
 'des',
 'schon',
 'seit',
 'Kurfürst',
 'Georg',
 'Wilhelm',
 'von',
 'der',
 'Familie',
 'von',
 'Briest',
 'bewohnten',
 'Herrenhauses',
 'zu',
 'Hohen-Cremmen',
 'fiel',
 'heller',
 'Sonnenschein',
 'auf',
 'die',
 'mittagsstille',
 'Dorfstraße,',
 'während',
 'nach',
 'der',
 'Park-',
 'und',
 'Gartenseite',
 'hin',
 'ein',
 'rechtwinklig',
 'angebauter',
 'Seitenflügel',
 'einen',
 'breiten',
 'Schatten',
 'erst',
 'auf',
 'einen',
 'weiß',
 'und',
 'grün',
 'quadrierten',
 'Fliesengang',
 'und',
 'dann',
 'über',
 'diesen',
 'hinaus',
 'auf',
 'ein',
 'großes,',
 'in',
 'seiner',
 'Mitte',
 'mit',
 'einer',
 'Sonnenuhr',
 'und',
 'an',
 'seinem',
 'Rande',
 'mit',
 'Canna',
 'indica',
 'und',
 'Rhabarberstauden',
 'besetzten',
 'Rondell',
 'warf.']

Wie viele Wörter gibt es insgesamt?

In [36]:
# print the length of the word list
len(words)

95930

Wie zu sehen ist, hat diese Art der "falschen" Tokenisierung den Nachteil, dass Satzzeichen nicht von Wörtern abgetrennt werden. \
Die Wortanzahl ist dementsprechend auch nicht akkurat. 

### Anzeigen von Worthäufigkeiten
Auf Grundlage dieser Wortliste kann trotzdem schon eine erste basale Häufigkeitenabfrage erfolgen. Dafür werden die Wörter zuerst gezählt. 

In [37]:
# Count the words with Counter and save the result to a variable
word_frequencies = Counter(words)

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Um die Häufigkeit nur mit Python abzufragen, kann folgende Zeile ausgeführt werden:
</details>

In [38]:
# 🚀 get the number of the word "Grippe" in the word frequencies 
word_frequencies["Luft"]

20

Dann kann die Häufigkeit abgefragt werden:

In [40]:
# Ensure Bokeh output is displayed in the notebook
output_notebook()

# Convert the dictionary to a JSON string to be passed to javascript
word_freq_json = json.dumps(word_frequencies)

# Create the text input widget
text_input = TextInput(value='', title="Geben Sie ein Wort ein:")

# Create a Div to display the frequency
frequency_display = Div(text="Häufigkeit: ")

# JavaScript callback to update the frequency display
# Only needed for graphical interface 
callback = CustomJS(args=dict(frequency_display=frequency_display, text_input=text_input), code=f"""
    var word = text_input.value.trim();

    // Parse the word frequency dictionary from Python
    var word_freq = {word_freq_json};

    var frequency = word in word_freq ? word_freq[word] : "Nicht gefunden";
    frequency_display.text = "Häufigkeit: " + frequency;
""")

text_input.js_on_change('value', callback)

# Layout and display
layout = column(text_input, frequency_display)
show(layout)

Loading BokehJS ...

## Annotation mit spaCy
Um eine präzisere Einteilung in Wörter zu erhalten (Tokenisierung) und um flektierte Wörter aufeinander abbildbar zu machen (Lemmatisierung), wird der Text im folgenden durch die Bibliothek [spaCy](https://spacy.io/) annotiert. Dafür werden folgende Schritte ausgeführt:
1. Das sprachspezifische Modell wird geladen. Wir arbeiten mit dem weniger akkuraten aber schnellsten spaCy Modell `de_core_news_sm`. 
2. Für eine erhöhte Annotationsgeschwindigkeit werden nur bestimmte Analysekomponenten geladen. Dies ist vor allem für größere Textmengen sinnvoll.
3. Der Text wird annotiert und die Token sowie die dazugehörigen Lemmata werden extrahiert.

### Sprachmodell laden
Das sprachspezifische Modell wird geladen. Es handelt sich dabei um das am wenigsten akkurate aber schnellste Modell. 

In [42]:
nlp = spacy.load('de_core_news_sm')

### Analysekomponenten auswählen
Es werden einige Analysekomponent wie z. B. das Aufteilen des Texts in Sätze (sentencizer) oder die [Named Entity Recognition](https://en.wikipedia.org/wiki/Named-entity_recognition) (ner) ausgeschlossen, da diese für die Tokenisierung und die Lemmatisierung nicht benötigt werden. Der Auschluss der Komponentnen erhöht die Annotationsgeschwindikgeit. 

In [44]:
disable_components = ['ner', 'attribute_ruler', 'sentencizer']

### Annotieren der Texte: Token, Lemma
Der ausgewählte Text wird mit spaCy annotiert und die Token sowie die dazugehörigen Lemmata werden extrahiert und in einer Tabelle gespeichert. Das Tabellenformat wurde gewählt, da sich darin gut relationale Daten speichern lassen.

In [55]:
# get the current time to display how long the annotation took
current = time()

# annotate with spacy
doc = nlp(text)

# extract tokens and lemmata, save them to a dictionary
text_annotated = {}
text_annotated['Token'] = [tok.text for tok in doc]
text_annotated['Lemma'] = [tok.lemma_ for tok in doc]
text_annotated['POS'] = [tok.pos_ for tok in doc]

# convert the dictionary to a dataframe 
text_annotated_df = pd.DataFrame(text_annotated)

# calculate how long the annotation and extraction took and print result
took = time() - current
print(f"Die Annotation hat {round(took, 2)} Sekunden gedauert.") 

Die Annotation hat 14.43 Sekunden gedauert.


Auszug aus der Tabelle, in der der annotierte Text gespeichert ist:

In [56]:
# print first five lines of the annotation
text_annotated_df.head()

,Token,Lemma,POS
0,Theodor,Theodor,PROPN
1,Fontane,Fontane,PROPN
2,\n\n,\n\n,SPACE
3,Effi,Effi,PROPN
4,Briest,Briest,PROPN


### Worthäufigkeit mit echter Tokenization   

Durch die Tokenisierung wurden z. B. Satzzeichen von Wörtern abgetrennt. An der Textlänge lässt sich dies schon erkennen. 

In [47]:
# get the lemmata 
text_tokenized = text_annotated_df.Lemma

# print the length
len(text_tokenized)

125666

Auf Grundlage des tokenisierten und lemmatisierten Texts, kann die Häufigkeitenabfrage erneut augeführt werden. Da durch die Lemmatisierung flektierte Wortformen auf die Grundformen zurückgeführt wurden, erwarten wir, dass die Häufigkeit einer Wortgrundform im Gegensatz zur vorherigen Abfrage erhöht ist. 

In [48]:
# Count the words with Counter and save the result to a variable
token_frequencies = Counter(text_tokenized)

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Um die Häufigkeit nur mit Python abzufragen, kann folgende Zeile ausgeführt werden:
</details>

In [50]:
# 🚀 get the number of the word "Grippe" in the word frequencies 
token_frequencies["Luft"]

29

In [51]:
# Ensure Bokeh output is displayed in the notebook
output_notebook()

# Convert the dictionary to a JSON string
tok_freq_json = json.dumps(token_frequencies)

# Create the text input widget
token_input = TextInput(value='', title="Geben Sie ein Wort ein:")

# Create a Div to display the frequency
token_frequency_display = Div(text="Häufigkeit: ")

# JavaScript callback to update the frequency display
# Only needed for graphical interface 
tok_callback = CustomJS(args=dict(frequency_display=token_frequency_display, text_input=token_input), code=f"""
    var tok = text_input.value.trim();

    // Parse the word frequency dictionary from Python
    var word_freq = {tok_freq_json};

    var frequency = tok in word_freq ? word_freq[tok] : "Nicht gefunden";
    frequency_display.text = "Häufigkeit: " + frequency;
""")

token_input.js_on_change('value', tok_callback)

# Layout and display
layout = column(token_input, token_frequency_display)
show(layout)

Loading BokehJS ...

## Annotation speichern
Um den annotierten Text zu speichern, wird zuerst der Dateiname festgelegt. Dafür wird die Dateiendung ersetzt von `.txt` zu `.csv`.

[CSV](https://en.wikipedia.org/wiki/Comma-separated_values) (comma-separated value) ist das Standardformat um tabellarische Daten im Klartext zu speichern. 

<details>
  <summary><b>Informationen zum Ausführen des Notebooks</b></summary>
Der Pfad zum Schreiben der Ergebnisse wird hier auf den selben Ordner gesetzt, in dem das Notebook liegt. So wird nicht von einer bestimmten Ordner-Struktur ausgegangen, wie in der Code-Zeile danach. Dort wird davon ausgeganen, dass auf der selben Höhe des Ordners, in dem das Notebook liegt, ein Ordner `data` existiert, in dem ein Ordner `csv` vorhanden ist. In dem Ordner `csv` wird die Annotation gespeichert. </br></br>
⚠️ Die nächste Zeile, in der der Pfad noch einmal gesetzt wird, muss übersprungen werden.
</details>

In [53]:
# set output path to current directory
output_dir = Path(r"../data/csv")
if not output_dir.exists():
    output_dir.mkdir()

output_path = output_dir / text_path.with_suffix(".csv").name

Der Text wird dann unter dem festgelegten Dateinamen gespeichert. 

In [54]:
# save the annotation as csv
text_annotated_df.to_csv(output_path, index=False)

## Prozess für das gesamte Korpus ausführen 

In [ ]:
def stream_texts_from_directory(corpus_filepaths: list[Path]) -> typing.Generator[str, None, None]:
    """A generator that yields texts from files in the file list one by one."""
    for filepath in corpus_filepaths:
            yield filepath.read_text(encoding="utf-8")

def process_corpus(corpus_dir: Path, output_dir: Path) -> None:
    """
    Reads files from corpus_dir, annotates the files with spacy and writes the result
    to the output_dir
    :param Path corpus_dir: The directory in which the txt files are saved
    :param Path output_dir: The directory in which the annotations are written to as csv
    """
    corpus_filepaths = [f for f in corpus_dir.iterdir() if f.is_file() and f.suffix == ".txt"]

    start = time()
    for filepath, doc in zip(corpus_filepaths, nlp.pipe(stream_texts_from_directory(corpus_filepaths), disable=disable_components)):
        print(filepath)
        # Save the token and lemma information to a dictionary
        text_annotated = {}
        text_annotated['Token'] = [tok.text for tok in doc]
        text_annotated['Lemma'] = [tok.lemma_ for tok in doc]
        text_annotated['POS'] = [tok.pos_ for tok in doc]
        annotation_df = pd.DataFrame(text_annotated)
        
        output_path = output_dir / filepath.with_suffix(".csv").name
        annotation_df.to_csv(output_path, index=False)
    end = time()
    
    print(f"""Processed {len(corpus_filepaths)} texts with spacy.
    Took {round((end - start)  / 60, 4)} minutes in total.""")

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Im folgenden werden alle Textdateien im Korpus heruntergeladen und gespeichert. Dafür sind folgende Schritte nötig:
<ol>
    <li>Es wird eine Liste erstellt, die die URLs zu den einzelnen Textdateien beinhaltet.</li>
    <li>Die Liste wird als txt-Datei gespeichert.</li>
    <li>Alle Dateien aus der Liste werden heruntergeladen und in dem Ordner <i>../data/txt</i> gespeichert.</li>
</ol>
Sollten die Dateien schon an einem anderen Ort vorhanden sein, können die Dateipfade zu den Ordnern angepasst werden. </br>
Des Weiteren wird der Ordner für die annotierten Dateien angelegt: <i>../data/csv</i>
</details>

In [ ]:
# 🚀 Create download list 
github_api_txt_dir_path = "https://api.github.com/repos/quadriga-dk/Text-Fallstudie-3/contents/data/corpus-of-german-fiction-txt"
txt_dir_info = requests.get(github_api_txt_dir_path).json()
url_list = [entry["download_url"] for entry in txt_dir_info]

# 🚀 Write download list as txt file
url_list_path = Path("github_txt_file_urls.txt")
with url_list_path.open('w') as output_txt:
    output_txt.write("\n".join(url_list))

In [ ]:
# ⚠️ Only execute, if you haven't downloaded the files yet!
# 🚀 Download all txt files – this step will take a while
! wget -i github_txt_file_urls.txt -P ../data/txt

In [ ]:
# Set path to corpus and output dir
corpus_dir = Path(r"../data/txt/")
output_dir = Path(r"../data/csv")

In [ ]:
# Read, annotate, write 
process_corpus(corpus_dir, output_dir)